# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark - 01_load_data ##

- Load listings.csv, calendar.csv, and reviews.csv
- Store cleaned datasets

## Code for loading listings ##

In [ ]:
import csv
from datetime import date

#default transformation
def transform_default(input_string):
    return input_string.upper().strip()

#transform a string date into string number of days after 1/1/08
start_date = date(2008, 1, 1)
def transform_date(date_string):
  if date_string == "":
    return ""
  elif "/" in date_string:
    month, day, year = date_string.split("/")
    end_date = date(2000 + int(year), int(month), int(day))
    return (end_date - start_date).days
  elif "-" in date_string:
    year, month, day = date_string.split("-")
    end_date = date(2000 + int(year), int(month), int(day))
    return (end_date - start_date).days
  else:
    raise ValueError("Cannot parse date " + date_string)

#transform price
def transform_price(price_string):
    if price_string == "":
        return ""
    elif len(price_string) > 3 and price_string[0] == "$" and price_string[-3:] == ".00":
        return price_string[1:-3].replace(",", "")
    else:
        raise ValueError("Cannot parse price " + price_string)

#transform zip by removing extended part
def transform_zip(zip_string):
    if len(zip_string) <= 5:
        return zip_string
    elif len(zip_string) == 10 and "-" in zip_string:
        #zip + 4 format
        return zip_string.rsplit("-")[0]
    else:
        #invalid (e.g CALL FOR DETAILS)
        return ""

#transform room type to int
bed_type_map = {'REAL BED': 0, 'FUTON': 1, 'PULL-OUT SOFA': 2, 'AIRBED': 3, 'COUCH': 4}
def transform_bed_type(input_string):
    return bed_type_map[input_string.upper()]

#transform room type to int
room_type_map = {'ENTIRE HOME/APT': 0, 'PRIVATE ROOM': 1, 'SHARED ROOM': 2}
def transform_room_type(input_string):
    return room_type_map[input_string.upper()]

#transform property type to int, combining rare entries into other = 5
property_type_map = {'APARTMENT': 0, 'HOUSE': 1, 'LOFT': 2, 'BED & BREAKFAST': 3, 'DORM': 4,
'': 5, 'CHALET': 5,    'EARTH HOUSE': 5, 'CAMPER/RV': 5, 'CABIN': 5, 'LIGHTHOUSE': 5,
'VILLA': 5,    'CAVE': 5, 'TREEHOUSE': 5, 'HUT': 5, 'OTHER': 5, 'BOAT': 5, 'CASTLE': 5, 'TENT': 5}
def transform_property_type(input_string):
    return property_type_map[input_string.upper()]

#list of all columns to keep and their cleanup function
column_mapping = {
    "id": transform_default,
    #"scrape_id"
    #"last_scraped"
    #"name"
    #"picture_url"
    #"host_id"
    #"host_name"
    "host_since": transform_date,
    #"host_picture_url"
    #"street"
    #"neighbourhood"
    #"neighbourhood_cleansed": transform_default,
    #"city"
    #"state"
    "zipcode": transform_zip,
    #"market"
    #"country"
    "latitude": transform_default,
    "longitude": transform_default,
    #"is_location_exact"
    "property_type": transform_property_type,
    "room_type": transform_room_type,
    "accommodates": transform_default,
    "bathrooms": transform_default,
    "bedrooms": transform_default,
    "beds": transform_default,
    "bed_type": transform_bed_type,
    #"square_feet"
    "price": transform_price,
    #"weekly_price"
    #"monthly_price"
    "guests_included": transform_default,
    #"extra_people"
    "minimum_nights": transform_default,
    "maximum_nights": transform_default,
    #"calendar_updated"
    "availability_30": transform_default,
    "availability_60": transform_default,
    "availability_90": transform_default,
    "availability_365": transform_default,
    #"calendar_last_scraped"
    "number_of_reviews": transform_default,
    "first_review": transform_date,
    "last_review": transform_date,
    "review_scores_rating": transform_default,
    "review_scores_accuracy": transform_default,
    "review_scores_cleanliness": transform_default,
    "review_scores_checkin": transform_default,
    "review_scores_communication": transform_default,
    "review_scores_location": transform_default,
    "review_scores_value": transform_default,
    "host_listing_count": transform_default
}

with open("../../Data/listings.csv", "rU") as original_data, open("../../Data/listings_clean.csv", "wb") as new_data:
    #create reader for data
    reader = csv.DictReader(original_data)

    #generate new headers
    headers = list(reader.fieldnames)
    new_headers = []
    for column in headers:
        if column in column_mapping.keys():
            new_headers.append(column)
    #move price to end
    new_headers.append(new_headers.pop(new_headers.index('price')))

    #create writer
    writer = csv.DictWriter(new_data, new_headers)
    writer.writeheader()

    for row in reader:
        new_row = {}
        #transform data
        for column, transformer in column_mapping.iteritems():
            new_row[column] = transformer(row[column])
        #write transformed data to new file
        writer.writerow(new_row)


## Code for loading calendar ##

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import csv
import string
from datetime import date, timedelta
%matplotlib inline

In [ ]:
#This cell flattens calendar table to grid of prices
#It takes input calendar.csv and outputs to a file calendar_grid.csv
#You should only need to run it once to generate the calendar_grid.csv file
def transform_price(price_string):
    if price_string == "":
        return ""
    elif len(price_string) > 3 and price_string[0] == "$" and price_string[-3:] == ".00":
        return price_string[1:-3].replace(",", "")
    elif len(price_string) > 1 and price_string[0] == "$":
        return price_string[1:].replace(",", "")
    else:
        raise ValueError("Cannot parse price " + price_string)

start_date = date(2015, 1, 1)
dates = [str(start_date + timedelta(days=i))[5:] for i in xrange(365)]

with open("../../Data/calendar.csv", "rU") as original_data, open("../../calendar_grid.csv", "w") as new_data:
    #create reader for data
    #we capture the second half of price in price2
    #file format is badly designed with commas in thousands e.g. $1,100
    reader = csv.DictReader(original_data, restkey="price2")

    #create writer
    writer = csv.DictWriter(new_data, ["listing_id"] + dates)
    writer.writeheader()

    new_row = None
    current_listing_id = None
    for row in reader:
        if current_listing_id != row["listing_id"]:
            #write previous row
            if new_row is not None:
                writer.writerow(new_row)
            #setup new row
            current_listing_id = row["listing_id"]
            new_row = {"listing_id": current_listing_id}
        #join price segments
        if "price2" in row:
            price = row["price"] + string.join(row["price2"])
        else:
            price = row["price"]
        #add date entry
        new_row[row["date"][5:]] = transform_price(price)
    #write final row
    writer.writerow(new_row)

## Code for loading reviews ##

In [ ]:
import pandas as pd
import numpy as np
import time
from langdetect import detect
import goslate
from guess_language import guess_language as guess_lan
import enchant

In [ ]:
# read data
reviews = pd.read_csv('../../Data/reviews.csv')

# check data
reviews.head()

In [ ]:
reviews['null_review']=np.zeros([reviews.shape[0],])
reviews['if_English']=np.zeros([reviews.shape[0],])

In [ ]:
def check_null(row):
    
    if pd.isnull(row['comments']) is True:
        return 1
    else:
        return 0

In [ ]:
# Clean up 'NaN' rows
reviews['null_review'] = reviews.apply(check_null, axis=1)
reviews = reviews[reviews['null_review'] == 0]
# Check reduced data size
print reviews.shape

In [ ]:
# check if review is English
def check_if_english1(row):
    in_str = row['comments']
    count_not_en = 0
    words = in_str.split()
    d = enchant.Dict("en_US")
    for idx in range(len(words)):
        if d.check(words[idx]) is False:
            count_not_en += 1
    if float(count_not_en)/len(words) > 0.3:
        return 0
    else:
        return 1

In [ ]:
# Check if reviews are English
reviews['if_English'] = reviews.apply(check_if_english1, axis=1)

In [ ]:
reviews_cleaned = pd.DataFrame(reviews[reviews['if_English']==1])
reviews_cleaned=reviews_cleaned.drop('null_review', axis = 1)
print reviews_cleaned.shape


In [ ]:
float(reviews_cleaned.shape[0])/277659

In [ ]:
reviews_cleaned[1000:1020]

In [ ]:
reviews_cleaned.to_csv('./Data/reviews_cleaned.csv', sep=',', index = False)